In [1]:
from dynamicSceneGenerator import DynamicSceneGenerator
from visualize import visualize, visualize_camera

In [2]:
# Generate dynamic scene with random tracks
dsg = DynamicSceneGenerator()
dsg.set_random_vessels(6)
dsg.generate_random_tracks()
vessels = dsg.get_vessels()
visualize(vessels)


![SegmentLocal](dynamicSceneExample.gif "Dynamic scene example")

In [76]:
import numpy as np

focal_length = 300*10**-6

px = py = 150*10**-6
position_WRF = np.array([500,200,0])
roll = 0
yaw = np.pi
pitch = 0
principal_point = (150,150)
image_bound = (300,300)
time_stamp = 166

In [77]:
from datatypes.virtualCamera import VirtualCamera

# (self, position_WRF, roll, yaw, pitch, focal_length, p_x, p_y, principal_point, skew=0):
camera = VirtualCamera(position_WRF, roll, yaw, pitch, focal_length, px, py, principal_point)

In [78]:
visualize_camera(time_stamp, camera, vessels, position_WRF[0]+50)

3.141592653589793
499.0
500.0


In [79]:
points = [vessel.calculate_3D_cornerpoints(time_stamp) for vessel in vessels]

In [80]:
projected_points = [camera.project_points(vessel_points) for vessel_points in points]

In [81]:
from visualize import plot_image

In [82]:
plot_image(projected_points, image_bound, time_stamp)

In [83]:
projected_points

[array([[149.99689711, 149.99718765,   1.        ],
        [149.99683658, 149.99705326,   1.        ],
        [149.99680023, 149.99703068,   1.        ],
        [149.99686215, 149.99716702,   1.        ],
        [149.99702123, 149.99718765,   1.        ],
        [149.99696311, 149.99705326,   1.        ],
        [149.99692822, 149.99703068,   1.        ],
        [149.99698766, 149.99716702,   1.        ]]),
 array([[149.99539221, 149.99563967,   1.        ],
        [149.99524206, 149.99548382,   1.        ],
        [149.99522799, 149.99549848,   1.        ],
        [149.99537902, 149.99565431,   1.        ],
        [149.99542908, 149.99563967,   1.        ],
        [149.99528012, 149.99548382,   1.        ],
        [149.99526616, 149.99549848,   1.        ],
        [149.99541598, 149.99565431,   1.        ]]),
 array([[149.99338658, 149.99426484,   1.        ],
        [149.99321931, 149.99420012,   1.        ],
        [149.99333406, 149.99431958,   1.        ],
        